In [1]:
#load necessary modules
import numpy as np
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt

In [3]:
from sklearn.metrics import confusion_matrix

In [4]:
print tf.__version__

1.3.0


In [5]:
from tensorflow.examples.tutorials.mnist import input_data

/home/lyn1x/software/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
#load mnist dataset
mnist = input_data.read_data_sets("mnist_data/", one_hot=True)

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


In [7]:
#a glips of how the dataset looks like
print("size of training set:",mnist.train.images.shape)

('size of training set:', (55000, 784))


In [8]:
print("size of test set", mnist.test.images.shape)

('size of test set', (10000, 784))


In [9]:
#define some global variables
image_flat_size = mnist.train.images.shape[1]

In [10]:
image_shape = (28,28)

In [11]:
mnist.train.images.imag

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
num_classes = 10

In [13]:
#check labels
mnist.train.labels[:5].argmax(axis=1)

array([7, 3, 4, 6, 1])

In [14]:
#plot iamges with labels

def plot_images(images, real_labels, pre_labels=None):
    assert len(images) == len(real_labels) == 9
    
    #creat a 3x3 canvas with subplots
    fig, axes = plt.subplots(3,3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)
    
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape((28,28)), cmap="binary")
    
        if pre_labels is None:
            xlabel = "True: {0}".format(real_labels[i])
        else:
            xlabel = "True: {0}, Predict{1}".format(real_label[i], pre_labels[i])
        
        ax.set_xlabel(xlabel)
        
        #remove ticks from the plot
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [15]:
#test plot function
plot_images(mnist.train.images[:9],mnist.train.labels[:9].argmax(axis=1))

In [16]:
#define placeholders(inputs)
x = tf.placeholder(tf.float32, [None, 784])
y_real = tf.placeholder(tf.float32, [None, 10])
y_real_labels = tf.placeholder(tf.int64,[None])

In [17]:
#define model variables, difference between get_variable and Variable,
#see:https://stackoverflow.com/questions/37098546/difference-between-variable-and-get-variable-in-tensorflow
W = tf.get_variable(name="weights", shape=[784,10], dtype=tf.float32)
b = tf.get_variable(name="bias", shape=[10], dtype=tf.float32)

In [18]:
# construct the model 
logits = tf.matmul(x, W) + b

In [19]:
pre_labels_onehot = tf.nn.softmax(logits)

In [21]:
pre_labels = tf.argmax(pre_labels_onehot,axis=1)

In [22]:
# cost function
cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=y_real)
cost = tf.reduce_mean(cross_entropy_loss)

In [23]:
#choose an optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(cost)

In [24]:
#performance measure
correct_prediction = tf.equal(y_real_labels, pre_labels)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [25]:
#create data_batchs
batch_size = 100

In [26]:
def get_next_batch(batch_size=batch_size):
    idx = np.arange(mnist.train.num_examples)
    np.random.shuffle(idx)
    start = 0
    end = start + batch_size
    has_next_batch = True
    
    while(has_next_batch):
        data_idx = idx[start:end]
        next_batch_images = mnist.train.images[data_idx]
        next_batch_labels = mnist.train.labels[data_idx]
        start += batch_size
        end += batch_size
        yield (next_batch_images,next_batch_labels)
        if end >= mnist.train.num_examples:
            has_next_batch = False

In [68]:
num_iteration = 500

In [69]:
#build the session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    assert num_iteration < 550
    for i in range(num_iteration):
        input_data = get_next_batch()
        x_batch, y_batch = input_data.next()
        #print(x.dtype, x_batch.dtype, y_real.dtype, y_batch.dtype)
        feed_dict = {x:x_batch, y_real:y_batch}
        sess.run(optimizer, feed_dict = feed_dict)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, \
                                        y_real:mnist.test.labels, \
                                        y_real_labels:np.argmax(mnist.test.labels,axis=1)})
        print(acc)

0.1375
0.1603
0.2031
0.2227
0.2553
0.2898
0.3269
0.3629
0.3938
0.4281
0.4683
0.4784
0.5119
0.5518
0.5568
0.5622
0.59
0.604
0.607
0.6115
0.6395
0.6496
0.6586
0.6693
0.6567
0.6759
0.694
0.7019
0.7025
0.712
0.722
0.7207
0.7246
0.7311
0.7366
0.744
0.7415
0.7394
0.7457
0.7538
0.742
0.7577
0.7593
0.7629
0.764
0.7661
0.767
0.7718
0.7802
0.7828
0.7847
0.7854
0.7887
0.7882
0.7908
0.795
0.7955
0.7951
0.7961
0.7945
0.7922
0.7929
0.793
0.793
0.7945
0.7953
0.7985
0.8019
0.8086
0.811
0.8116
0.8144
0.8151
0.8171
0.8168
0.8169
0.82
0.819
0.8167
0.8184
0.8199
0.8217
0.8246
0.8229
0.8281
0.8301
0.827
0.828
0.8272
0.8277
0.83
0.8295
0.8305
0.8334
0.8345
0.8341
0.8338
0.8355
0.8348
0.8333
0.8369
0.84
0.8388
0.8389
0.8362
0.8377
0.836
0.8355
0.8356
0.838
0.8391
0.8418
0.8431
0.8423
0.8424
0.84
0.8429
0.8417
0.8405
0.8419
0.843
0.8441
0.844
0.8468
0.8467
0.8481
0.8483
0.8485
0.8489
0.8456
0.8457
0.8484
0.8487
0.8471
0.8485
0.8526
0.8496
0.85
0.8498
0.8512
0.8515
0.85
0.8497
0.8507
0.8497
0.853
0.8521
0.8551

In [30]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [31]:
def optimize(num_iteration):
    for i in range(num_iteration):
        x_batch, y_batch = mnist.train.next_batch(100)
        feed_dict_train = {x:x_batch, y_real:y_batch}
        sess.run(optimizer, feed_dict=feed_dict_train)

In [32]:
optimize(1)

In [37]:
def print_acc():
    acc = sess.run(accuracy, feed_dict={x:mnist.test.images, \
                                        y_real:mnist.test.labels, \
                                        y_real_labels:np.argmax(mnist.test.labels,axis=1)})
    print("accuracy on testset: {0:.2%}".format(acc))

In [38]:
print_acc()

accuracy on testset: 12.85%


In [55]:
def print_confusion_matrix():
    real_labels = np.argmax(mnist.test.labels, axis=1)
    pred_labels = sess.run(pre_labels, feed_dict={x:mnist.test.images, \
                                                   y_real:mnist.test.labels})
    #print(real_labels, pred_labels)
    confusion_mat = confusion_matrix(y_true=real_labels, y_pred=pred_labels)
    print(confusion_mat)
    plt.imshow(confusion_mat, interpolation="nearest", cmap=plt.cm.Blues)
    plt.tight_layout()
    plt.colorbar()
    plt.xlabel('predict')
    plt.ylabel('True')
    plt.show()

In [56]:
print_confusion_matrix()

[[286   0 263   0   0 120  41 243  24   3]
 [ 55   0 368   2   0 115  17 578   0   0]
 [330   1 313   6   2  71  20 232  38  19]
 [169   1 657   0   3  31  45  96   4   4]
 [188  32 164  75   4  92   8 224 110  85]
 [274  12 404   0   3  35  53 107   3   1]
 [312  20 192   4  19  31 112 141  78  49]
 [ 57   3 340  27   0  62   7 475  14  43]
 [203   0 358   0   6  74  44 262  11  16]
 [201  10 252  18   0  34   8 420  17  49]]


In [ ]:
def plot_example_errors():